In [1]:
import numpy as np
import itertools
import copy

from scipy.optimize import fsolve
from scipy.special import gamma

In [2]:
class City(object):
    
    def __init__(self, width, height, hospital_count, subregion_count):
        
        self.width = width
        self.height = height
        #for simplicity, subregion_count = base_count
        
        hospital_x = np.linspace(0, self.width, num = hospital_count + 2)
        self.hospital_location = [(hospital_x[i+1], self.height/2) for i in range(hospital_count)]
        
        self.subregion_count = subregion_count + (subregion_count % 2)
        subregion_x = np.linspace(0, self.width, num = int(self.subregion_count/2 + 2))
        subregion_y = np.linspace(0, self.height, num = 4)
        base_location = [[(subregion_x[i+1], subregion_y[j+1]) for j in range(2)] for i in range(int(self.subregion_count/2))]
        self.base_location = list(itertools.chain(*base_location))
        

In [3]:
def gen_call_arrival_loc(city):
    return (np.random.uniform(0, city.width),np.random.uniform(0, city.height))

In [4]:
def gen_call_arrival_time():
    return np.random.exponential(1)

In [5]:
def gen_atlocation_service_time():
    return np.random.exponential(12)

In [6]:
def weibull_param_relationship(scale, mean, stddev):
    return stddev**2/mean**2 - gamma(1 + 2/scale)/(gamma(1+1/scale)**2) + 1

def get_weibull_parameters(mean = 30, stddev = 13):
    scale = fsolve(weibull_param_relationship, 1,args=(mean, stddev))
    shape = mean / gamma(1+1/scale)
    
    return scale, shape

def gen_athospital_service_time():
    scale, shape = get_weibull_parameters(mean = 30, stddev = 13)
    return scale * np.random.weibull(shape)

In [7]:
class Call(object):
    
    def __init__(self, location, interarrival_time, priority = 0):
        
        self.location = location
        
        #initialize the call to be unassigned
        #-1-unassigned, 0 to (N-1) assigned ambulance index 
        self.status = -1
        self.interarrival_time = interarrival_time
        self.priority = priority

In [8]:
def get_distance(location1, location2):
    
    distance = np.abs(location1[0] - location2[0]) + np.abs(location1[1] - location2[1])
    return distance

In [9]:
def get_ambulance_travel_time(distance, speed = 10):
    return distance/speed

In [10]:
class Ambulance(object):
    
    def __init__(self, base):
        
        #fix the ambulance's home base
        self.base = base
        
        #initialize the ambulance idle at home base
        
        #status code: 0-idle at base, 1-going to scene of call, 2-serving at scene of call, 3-going to hospital
        #4-transferring patient at hospital, 5-returning to base
        self.status = 0
        self.origin = base
        self.destination = base # if destination = origin, ambulance is stationary
        self.time = 0
        self.endtime = np.inf
        
    def update_status(self, status, origin, destination, time, endtime):
        self.status = status
        
        self.origin = origin
        self.destination = destination
        self.time = time
        self.endtime = endtime
        
    def redployment(self):
        
        #current heuristic: return to home base
        distance = get_distance(self.origin, self.base)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(5, self.origin, copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
        
    def assign_to_call(self, call, time):
        
        distance = get_distance(self.origin, call.location)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(1, self.origin, call.location, time, time + travel_time)
        
        call.status = self
        
        
    def reach_call_location(self, call, callList):
    
        atlocation_servicetime = gen_atlocation_service_time()
        self.update_status(2, call.location, call.location, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + atlocation_servicetime)

        callList.remove(call)
    
    def transport_to_hospital(self, city):
        
        #transport to nearest hospital
        
        hospital_list = city.hospital_location
        
        min_distance = city.width + city.height
        nearest_hospital = hospital_list[0]
        for hospital in hospital_list:
            distance = get_distance(self.origin, hospital)
            if distance < nearest_distance:
                min_distance = distance
                nearest_hospital = hospital
        
        travel_time = get_ambulance_travel_time(min_distance)
        self.update_status(3, self.origin, hospital, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
    def reach_hospital(self):
        hospital_servicetime = gen_athospital_service_time()
        self.update_status(4, copy.deepcopy(self.destination), copy.deepcopy(self.destination), 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + hospital_servicetime)




In [11]:
def arrival(ambulanceList, callList, time_arrival, M):
    if len(callList) >= M:
        return
    
    call = Call(gen_call_arrival_loc(city), gen_call_arrival_time())
    callList.append(call)
    
    min_distance = 15
    assign = -1
    index = 0
    
    nearest_distance = 15
    for ambulance in ambulanceList:

        if ambulance.status == 0:
            distance = get_distance(ambulance.origin, call.location)
            if distance < nearest_distance:
                nearest_distance = distance
                assign = index

        index = index + 1
    
    if assign > -1:
        # when the call arrives, there are ambulances idle at base, so assign the call to the nearest ambulance
        ambulanceList[assign].assign_to_call(call, time_arrival)
    
    time_arrival = time_arrival + call.interarrival_time
    
    return ambulanceList, callList, time_arrival

In [12]:
def get_next_event(time_arrival, ambulanceList, callList, city, M):
    
    ambulanceEndTime_min = np.inf
    index_min = -1
    index = 0
    for ambulance in ambulanceList:
        if ambulance.endtime < ambulanceEndTime_min:
            ambulanceEndTime_min = ambulance.endtime
            index_min = index
        
        index = index + 1
    
    next_event_time = min(time_arrival, ambulanceEndTime_min)
    
    if next_event_time == time_arrival:
        ambulanceList, callList, time_arrival = arrival(ambulanceList, callList, time_arrival, M)
    else:
        if ambulanceList[index_min] == 1:
            call = callList[0]
            ambulanceList[index_min].reach_call_location(call, callList)
        elif ambulanceList[index_min] == 2:
            ambulanceList[index_min].transport_to_hospital(city)
        elif ambulanceList[index_min] == 3:
            ambulanceList[index_min].reach_hospital()
        elif ambulanceList[index_min] == 4:
            
            if len(callList) == 0:
                ambulanceList[index_min].redployment()
            else:
                assign = -1
                index = 0
                while assign == -1 and index < M:

                    if callList[index].status == -1:
                        assign = index

                    index = index + 1
                
                if index == M:
                    # calls in callList have all been assigned with an ambulance
                    ambulanceList[index_min].redployment()
                else:
                    ambulanceList[index_min].assign_to_call(callList[assign], next_event_time)
    
    return time_arrival, ambulanceList, callList

In [13]:
city = City(10, 5, 2, 4)

In [14]:
ambulanceList = [Ambulance(city.base_location[0]), Ambulance(city.base_location[1])]

In [15]:
callList = []

In [16]:
time_arrival = 0

In [17]:
time_arrival, ambulanceList, callList = get_next_event(time_arrival, ambulanceList, callList, city, 3)

In [18]:
time_arrival

1.4794732990185147

In [19]:
ambulanceList

[<__main__.Ambulance at 0x1629d3fe668>, <__main__.Ambulance at 0x1629d3fee48>]

In [20]:
ambulanceList[0].status

1

In [21]:
callList[0].status